- This code has written by Ahmadreza Attarpour (a.attarpour@mail.utoronto.ca)
- It's a practice to check how we can use LangChain/LangGraph to build tools and agents

-------------------------------------

# Self-Ask With Search Agent

- devide/break down the input into multiple intermediate questions and find the final answer
- This approach allows the AI to ask itself follow up questions when it doesn't immediately know the answer. It performs a recursive questioning process to break down complex queries into smaller, more manageable ones.
- once the AI generates these follow-up questions, it performs an external serach using tools to gather info before final answer.


In [18]:
import os

In [19]:
# load the environment variables from the .env file
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
# Read the variable from the environment and set it as a env variable
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
if GOOGLE_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
if LANGCHAIN_API_KEY:
    os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
if LANGCHAIN_PROJECT:
    os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
if os.getenv("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
if os.getenv("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
if os.getenv("SERPER_API_KEY"):
    os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [22]:
# load the LLM and embeddings from langchain_google_genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# This is a sentence-transformers model: It maps sentences & paragraphs to a
# 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.
# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
# from langchain_groq import ChatGroq
# llm = ChatGroq(model="Gemma2-9b-It")
# llm = ChatGroq(model="llama3-70b-8192")

/Users/ahmadreza/anaconda3/envs/aa_agent/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
from langchain.agents import create_self_ask_with_search_agent, AgentExecutor, tool, Tool
from langchain import hub
from langchain_community.tools.tavily_search import TavilyAnswer
from langchain.utilities import GoogleSerperAPIWrapper


In [24]:
# create the search tool
search = GoogleSerperAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="Useful for when you need to answer questions with search.",
        verbose=True
    )
]

In [25]:
# create prompt
prompt = hub.pull("hwchase17/self-ask-with-search")
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [26]:
# build the agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

In [27]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

In [28]:
agent_executor.invoke({
    "input": "which asian country has the largest population?"
})



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the population of India?1.451 billion (2024)Follow up: What is the population of China?China: Country in East Asia. China, officially the People's Republic of China, is a country in East Asia. With a population exceeding 1.4 billion, it is the second-most populous country after India, representing 17.4% of the world population. China Capital: Beijing. The current population of China is 1,415,782,819 as of Tuesday, August 5, 2025, based on Worldometer's elaboration of the latest United Nations data1. China ... By one estimate, in 2024 China's population stood at about 1.408 billion, down from the 1.412 billion recorded in the 2020 census. According to the 2020 census, ... World Population ; China, 1,407,181,209 ; United States, 342,034,432 ; Indonesia, 283,587,097 ; Pakistan, 257,047,044 ; Nigeria, 242,794,751. China is a country in Asia. The population in China was 1,408,975,000 in 2024. The nominal GDP per capita in China

{'input': 'which asian country has the largest population?',
 'output': 'India',
 'intermediate_steps': [(AgentAction(tool='Intermediate Answer', tool_input='What is the population of India?', log='Yes.\nFollow up: What is the population of India?'),
   '1.451 billion (2024)'),
  (AgentAction(tool='Intermediate Answer', tool_input='What is the population of China?', log='Follow up: What is the population of China?'),
   "China: Country in East Asia. China, officially the People's Republic of China, is a country in East Asia. With a population exceeding 1.4 billion, it is the second-most populous country after India, representing 17.4% of the world population. China Capital: Beijing. The current population of China is 1,415,782,819 as of Tuesday, August 5, 2025, based on Worldometer's elaboration of the latest United Nations data1. China ... By one estimate, in 2024 China's population stood at about 1.408 billion, down from the 1.412 billion recorded in the 2020 census. According to the

# custom tool

In [7]:
# custom tool
@tool
def get_employee_id(name: str) -> str:
    """Get the employee ID for a given name."""

    fake_employees = {
        "Alice": "12345",
        "Bob": "67890",
        "Charlie": "54321",
        "David": "98765",
        "Eve": "11223",
        "Frank": "44556",
        "Grace": "77889",
        "Heidi": "33445",
        "Ivan": "66778",
        "Judy": "88990"
    }
    # This is a mock implementation, replace with actual logic
    return fake_employees.get(name, "Employee not found")

@tool
def get_employee_salary(name: str) -> str:
    """Get the employee salary for a given name."""

    fake_salaries = {
        "Alice": "$100,000",
        "Bob": "$120,000",
        "Charlie": "$110,000",
        "David": "$130,000",
        "Eve": "$140,000",
        "Frank": "$150,000",
        "Grace": "$160,000",
        "Heidi": "$170,000",
        "Ivan": "$180,000",
        "Judy": "$190,000"
    }
    # This is a mock implementation, replace with actual logic
    return fake_salaries.get(name, "Salary not found")

In [8]:
# create the tools
tools = [
    get_employee_id,
    get_employee_salary
]

In [9]:
# get the prompt
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [10]:
# create the agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools, 
    verbose=True,
    return_intermediate_steps=True
)

In [13]:
# invoke the agent
agent_executor.invoke({
    "input": "what's the salary of Alice?"
})



> Entering new AgentExecutor chain...
Thought: I need to find Alice's employee ID first and then use it to get her salary.
Action: get_employee_id
Action Input: Alice12345Thought: Now that I have Alice's employee ID, I can get her salary.
Action: get_employee_salary
Action Input: Alice$100,000Thought: I now know the final answer
Final Answer: Alice's salary is $100,000 


> Finished chain.


{'input': "what's the salary of Alice?",
 'output': "Alice's salary is $100,000",
 'intermediate_steps': [(AgentAction(tool='get_employee_id', tool_input='Alice', log="Thought: I need to find Alice's employee ID first and then use it to get her salary.\nAction: get_employee_id\nAction Input: Alice"),
   '12345'),
  (AgentAction(tool='get_employee_salary', tool_input='Alice', log="Thought: Now that I have Alice's employee ID, I can get her salary.\nAction: get_employee_salary\nAction Input: Alice"),
   '$100,000')]}